# Process LED data

Load MIDAS data with LED triggers. Make PSD and pulse templates, then run OF

Changelog

Oct 30 2024: First version

In [1]:
import os
import sys
import glob
sys.path.append('..')
from arqpy import *

In [2]:
# data + channels to use
rns = [16346,16347,16348,16349,16350,16351,16352,16353,16354]

datadir = '/fs/ddn/sdf/group/supercdms/data/CDMS/SLAC/OLAF12/'
allfns = [sorted(glob.glob(datadir+f'RUN{rn}_DUMP000*.mid.gz')) for rn in rns]
chs = ['PBS2','PFS2'] # channels to use (which also serve as dictionary keys)
names = ['NFC2 A','NFC2 B']
resultdir = './results/'
os.makedirs(resultdir,exist_ok=True)

# trigger options
trigger_mode = 3 # 0 - threshold, 1 - filter+threshold, 2 - random, 3 - external/LEMO
randomrate = 0 # random triggers to add per trace
offset = -0.0042 # fixed offset s.t. pulse = LEMO trigger - offset, sec
psd_offset = 0.0023 # offset for aligning traces BETWEEN pulses, for PSD-making, sec

# OF,trace options
maxchi2freq = 1e4 # Hz
pretrig     = 1000 # bins
posttrig    = 4000 # bins


# general
fsamp = 625000 # Hz
ADC2A = 1/2**16 *8/5e3 /2.4/4 # 16-bit ADC, 8V range, 5kOhm R_FB, 2.4 turn ratio, gain = 4
det   = 1 # MIDAS Det01

tracelen = pretrig + posttrig # trace used for RQ processing
psdfreq = np.fft.rfftfreq(tracelen,1/fsamp)
# convert time to number of bins
trigger_offsets = {} 
psd_offsets = {}
for ch in chs:
    trigger_offsets[ch] = offset*fsamp
    psd_offsets[ch] = psd_offset*fsamp

In [3]:
os.remove(f'{resultdir}results.csv')
maxA = 2e-7
for i in range(len(rns)):
    rn = rns[i]
    fns = allfns[i]

    # track: template_amp template_t1 template_t2 template_integral OF0_A_med OF0_A_sig
    csvdata = {}
    for ch in chs:
        csvdata[ch] = []
        
    # instantiate Reduced Quantity object, without templates
    rq = RQ(data=fns, chs=chs, ch_names=names, detector=det,
            fsamp=fsamp, ADC2A=ADC2A,
            pretrig=pretrig, posttrig=posttrig, 
            PSDs=None, pulse_templates=None, # !
            maxchi2freq=maxchi2freq)

    # calculate PSDs, then plot
    rq.runTrigger(mode=trigger_mode, chs=chs, randomrate=randomrate, trigger_offsets=psd_offsets)
    psds = makePSDs(rq.traces,chs=chs,nbins=tracelen,ntraces=len(rq.traces[chs[0]]),fsamp=fsamp)
    plotPSDs(psds,fsamp=fsamp,tracelen=tracelen)
    plt.savefig(f'{resultdir}{rn}_psds.png')
    plt.close('all')

    # get average pulse, but then fit anyways
    #emp_templates = {} # empirical templates
    templates = {} # analytical fits that will be used
    fig,axes = plt.subplots(2,1)
    for ch in chs:
        template = rq.averageTrigPulse(fns[:1],ch,150,offset)
        template -= np.mean(template[:100]) # "pedestal" subtraction
        template = lpf(template,3e4,10)
        #emp_templates[ch] = template / np.max(template) # unused

        # 2-pole fit
        t = np.arange(len(template))/fsamp
        x0 = (np.max(template), -4.3,-3.5, t[np.argmax(template)], 0)
        mle = fitPulse(t,template,2,x0=x0)
        templates[ch] = pulse2(t,*mle[1:-1]) # amplitude = 1.0
        
        fit = mle[0]*pulse2(t,*mle[1:-1]) # amplitude = physical units
        csvdata[ch] += [mle[0],mle[1],mle[2],np.sum(fit)/fsamp]
        
        c = MIDAScolors[MIDASchs.index(ch)]
        axes[0].plot(t,template,color=c,alpha=0.3)
        axes[0].plot(t,fit+mle[-1],color=c,zorder=10,label=ch)
        f,psd = sqrtpsd(t,template)
        axes[1].loglog(f,psd,color=c,alpha=0.3)
        f,template_psd = sqrtpsd(t,fit)
        axes[1].loglog(f,template_psd,color=c)
    axes[0].legend(loc=1)
    axes[0].set_xlabel('time (s)')
    axes[0].set_ylabel('current (A)')
    axes[1].grid()
    axes[1].grid(which='minor',alpha=0.2)
    axes[1].set_xlim(min(psdfreq[psdfreq>0]),max(psdfreq))
    axes[1].set_ylim(1e-12,2.5e-8)
    axes[1].set_xlabel('frequency (Hz)')
    axes[1].set_ylabel(r'noise (A/$\sqrt{Hz}$)')
    plt.savefig(f'{resultdir}{rn}_templates.png')
    plt.close('all')
    
    # make OF (for RQs AND triggering)
    rq.setPSD(psds,ch)
    rq.setTemplates(templates,ch)
    rq.makeOF()

    res = rq.getTheoryRes()
    for ch in chs:
        csvdata[ch].append(res[ch])

    # run trigger
    rq.runTrigger(mode=trigger_mode, chs=chs, randomrate=randomrate, trigger_offsets=trigger_offsets)

    # Run optimal filter, etc. to derive RQs
    rq.processTraces()
    
    # results are stored in rq.results
    RQs = rq.results
    # could save this as a .pkl


    # RQ plots
    # spectra
    bins = np.linspace(-0.5e-8,maxA,200)
    for ch in chs:
        amps = RQs[f'OF0_A_{ch}']
        c = MIDAScolors[MIDASchs.index(ch)]
        plt.hist(amps,bins=bins,histtype='step',color=c)
        csvdata[ch] += [np.median(amps),
                        (np.percentile(amps,50+68.3/2)-np.percentile(amps,50-68.3/2))/2 ]
    plt.title(f'RUN {rn} Spectrum')
    plt.xlabel('OF0_A')
    plt.ylabel('events')
    
    plt.savefig(f'{resultdir}{rn}_spectra.png')
    plt.close('all')

    # chi2 vs amplitude plots
    for ch in chs:
        a = RQs[f'OF0_A_{ch}']
        chi2=RQs[f'OF0_chi2_{ch}']
        bins = (np.linspace(0,maxA,200),np.linspace(0,np.max(chi2),200))
        plt.hist2d(a,chi2,bins=bins,norm=LogNorm())
        plt.axhline(2*np.sum(psdfreq<maxchi2freq),color='k',alpha=0.3)
        plt.xlabel(f'OF0_A_{ch}')
        plt.ylabel(f'OF0_chi2_{ch}')
        plt.colorbar()
        # we want one of these per channel
        plt.savefig(f'{resultdir}{rn}_{ch}_chi2.png')
        plt.close('all')

    # partition
    ch0,ch1 = chs
    amp0 = RQs[f'OF0_A_{ch0}']
    amp1 = RQs[f'OF0_A_{ch1}']
    
    bins = (np.linspace(0,maxA,200),np.linspace(0,maxA,200))
    
    plt.hist2d(amp0,amp1,bins=bins,norm=LogNorm())
    plt.xlabel(f'OF0_A_{ch0} (A)')
    plt.ylabel(f'OF0_A_{ch1} (A)')
    plt.gca().set_aspect('equal')
    plt.savefig(f'{resultdir}{rn}_partition.png')
    plt.close('all')

    with open(f'{resultdir}results.csv','a') as fp:
        line = f'{rn},'
        for ch in chs:
            line += ','.join(['{0}'.format(val) for val in csvdata[ch]])
            if ch != chs[-1]:
                line += ','
        fp.write(line+'\n')

IndentationError: unexpected indent (1617513656.py, line 132)